In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import urllib.parse
from sklearn import tree
from sklearn import metrics
import io
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

In [11]:
normal_file_raw = '../dataset/web-application-attacks-datasets/csic_2010/normalTrafficTraining.txt'
anomaly_file_raw = '../dataset/web-application-attacks-datasets/csic_2010/anomalousTrafficTest.txt'

normal_file_parse = '../dataset/web-application-attacks-datasets/csic_2010/normalRequestTraining.txt'
anomaly_file_parse = '../dataset/web-application-attacks-datasets/csic_2010/anomalousRequestTest.txt'

In [52]:
def parse_file(file_in, file_out):
    fin = open(file_in)
    fout = io.open(file_out, "w", encoding="utf-8")
    lines = fin.readlines()
    res = []
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith("GET"):
            res.append("GET" + line.split(" ")[1])
        elif line.startswith("POST") or line.startswith("PUT"):
            url = line.split(' ')[0] + line.split(' ')[1]
            j = i
            while i != j:
                if lines[j].startswith("Content-Length"):
                    break
                j += 1
            j += 1
            data = lines[i].strip()
            url += '?' + data
            res.append(url)
    for line in res:
        line = urllib.parse.unquote(line).replace('\n','').lower()
        fout.writelines(line + '\n')
    print ("finished parse ",len(res)," requests")
    fout.close()
    fin.close()


In [53]:
def loadData(file):
    with open(file, 'r', encoding="utf8") as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if (len(d) > 0):
            result.append(d)
    return result

In [54]:
parse_file(normal_file_raw,normal_file_parse)
parse_file(anomaly_file_raw,anomaly_file_parse)

finished parse  36000  requests
finished parse  16327  requests


In [55]:

bad_requests = loadData('../dataset/web-application-attacks-datasets/csic_2010/anomalousRequestTest.txt')
good_requests = loadData('../dataset/web-application-attacks-datasets/csic_2010/normalRequestTraining.txt')

all_requests = bad_requests + good_requests
yBad = [1] * len(bad_requests)
yGood = [0] * len(good_requests)
y = yBad + yGood

In [56]:
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3))
X = vectorizer.fit_transform(all_requests)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)

In [57]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score_test = metrics.accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)

print ("Score Logistic Regression :",score_test)
print ("Confusion Matrix: ")
print (matrix)

Score Logistic Regression : 0.864131473342251
Confusion Matrix: 
[[3567   22]
 [ 689  955]]
